# Permite extraer las imágenes de los diagramas de un documento DOCx o PDF

0) Preparar ambiente e instalar paquetes:

In [1]:
#@title Instala paquetes 

# para procesar PDFs [https://pymupdf.readthedocs.io/en/latest/]
!pip install PyMuPDF

print("\n")

# para procesar DocX [https://pypi.org/project/docx2python/]
!pip install docx2python

     |████████████████████████████████| 6.4 MB 12.6 MB/s 




1) Cargar librerías:

In [2]:
#@title Cargar Librerías
import os

import fitz
from docx2python import docx2python

print("Librerías cargadas.")

Librerías cargadas.


2) Montar el Drive:

In [3]:
#@title Montar Google Drive
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


3) Lleva a cabo el procesamiento de los documentos:

In [9]:
#@title Procesa los documentos y graba las imágenes correspondientes

drive_path = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas' #@param {type:"string"}
drive_origen_documentos = '/Diagramas-Enunciados/Originales' #@param {type:"string"}
drive_destino_diagramas = '/Diagramas-Enunciados' #@param {type:"string"}
origen_dir_path = drive_path + drive_origen_documentos
destino_dir_path = drive_path + drive_destino_diagramas

renombrar_documentos_procesados = True #@param {type:"boolean"}

# levanta y procesa los documentos
process_FileNames = [ fn for fn in os.listdir( origen_dir_path ) if fn.endswith('.pdf') or fn.endswith('.docx') ]
print("\n> Documentos a procesar: ", len(process_FileNames))
for fn in process_FileNames:
  print("\n>", fn, ":")

  # determina nombre y tipo de archivo
  fnFull = origen_dir_path+"/"+fn
  nombre, tipo = os.path.splitext(fn)
  tipo = tipo.lower()
  cantImgExtraidas = 0

  if "pdf" in tipo:
    # procesa PDF
    res = fitz.open(fnFull)
    for i in range(len(res)):
      print("   Página ", i, "- imágenes encontradas: ", len(res.getPageImageList(i)))
      for img in res.getPageImageList(i):
          xref = img[0]
          pix = fitz.Pixmap(res, xref)
          imagFN = "/%s_P%sR%s.png" % (nombre, i, xref)
          print("\t"+imagFN, sep=' ')
          cantImgExtraidas = cantImgExtraidas + 1
          if pix.n < 5:       # this is GRAY or RGB
              pix.writePNG(destino_dir_path + imagFN)
          else:               # CMYK: convert to RGB first
              pix1 = fitz.Pixmap(fitz.csRGB, pix)
              pix1.writePNG(destino_dir_path + imagFN)
              pix1 = None
          pix = None
    # renombra archivo como procesado
    if renombrar_documentos_procesados and (cantImgExtraidas>0):
      os.rename(fnFull, fnFull+"-LISTO")

  elif "doc" in tipo:    
      # procesa DOCx
      res = docx2python(fnFull)
      print("  imágenes encontradas: ", len(res.images.items()))      
      for iname, dimage in res.images.items():
        imagFN = "/%s_%s" % (nombre, iname)
        print("\t"+imagFN, sep=' ')
        cantImgExtraidas = cantImgExtraidas + 1
        imgFile = open(destino_dir_path + imagFN, 'wb')
        imgFile.write(dimage)
        imgFile.close()
      # renombra archivo como procesado
      if renombrar_documentos_procesados and (cantImgExtraidas>0):
        os.rename(fnFull, fnFull+"-LISTO")
  else:
    print("Tipo de archivo desconocido: ", tipo)


> Documentos a procesar:  56

> Cinef - TPC3 - Grupo After Study.pdf :
   Página  0 - imágenes encontradas:  0
   Página  1 - imágenes encontradas:  1
	/Cinef - TPC3 - Grupo After Study_P1R16.png
   Página  2 - imágenes encontradas:  1
	/Cinef - TPC3 - Grupo After Study_P2R21.png
   Página  3 - imágenes encontradas:  1
	/Cinef - TPC3 - Grupo After Study_P3R24.png

> Cinef - TPC 3 - Pistachos Scicilianos.pdf :
   Página  0 - imágenes encontradas:  0
   Página  1 - imágenes encontradas:  1
	/Cinef - TPC 3 - Pistachos Scicilianos_P1R18.png
   Página  2 - imágenes encontradas:  1
	/Cinef - TPC 3 - Pistachos Scicilianos_P2R21.png

> Cinef - TPC3 - GrupoApollo.pdf :
   Página  0 - imágenes encontradas:  0
   Página  1 - imágenes encontradas:  1
	/Cinef - TPC3 - GrupoApollo_P1R18.png

> TPC3 Cinef  Los Pibardos.pdf :
   Página  0 - imágenes encontradas:  0
   Página  1 - imágenes encontradas:  1
	/TPC3 Cinef  Los Pibardos_P1R26.png

> Cinef - TPC3 - Le Bird.pdf :
   Página  0 - imágenes enco

In [8]:
#@title Volver a renombrar documentos procesados

sacar_marca_listos = False #@param {type:"boolean"}

if sacar_marca_listos:
  process_FileNames = [ fn for fn in os.listdir( origen_dir_path ) if fn.endswith('-LISTO')  ]
  print("\n> Documentos a renombrar: ", len(process_FileNames))
  for fn in process_FileNames:
    fnFull = origen_dir_path+"/"+fn
    os.rename(fnFull, fnFull[:len(fnFull)-len("-LISTO")])
